# 50인 데이터 매칭 알고리즘

사용 언어: Python

가치관 질문은 사용하지 않는다고 가정하고 우선 제외한 상태로 작성했습니다.

현재 구현 버전에서는 성별/연령 차이를 구현하지 않았습니다.

(09/09 수정: 매칭 알고리즘 보완)

(09/15 수정: 매칭 알고리즘 재수정, 2인씩 매칭하도록 고침. 전체 응답차 내에서 순위를 정해서 내려오는 방식으로 수정, 성별 및 연령 도입)

## 데이터 정제

데이터 분석 및 정리에 필요한 라이브러리(pandas, numpy)를 불러옵니다.

주어진 엑셀 파일에서 미리 사용하지 않을 열(타임스탬프, 가치관 질문, 빈 열 등)을 삭제하고, CSV 파일형식으로 변환하여 불러옵니다.
dataCount 변수에 현재 데이터 개수(응답 인원수)를 저장합니다.

In [2]:
import pandas as pd
import numpy as np

In [2]:
# set of column names
headerNames = ["q1", "q2", "q3", "q4", "q5", "q6", "q7","q8", "q9", "q10", "gender", "age", "polAlign"]

# read csv-converted data from excel file
# current data is cleaned to exclude empty and/or meaningless columns
answerData = pd.read_csv('data/50data_answeronly.csv', header =0, names=headerNames)

# save number of entries
dataCount = len(answerData.index)

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,③ 중립,① 매우 찬성,② 찬성,④ 반대,② 찬성,① 매우 찬성,③ 중립,② 찬성,③ 중립,② 찬성,남,40,2
1,① 매우 찬성,③ 중립,② 찬성,② 찬성,⑤ 매우 반대,④ 반대,② 찬성,④ 반대,③ 중립,① 매우 찬성,여,25,2
2,② 찬성,⑤ 매우 반대,③ 중립,② 찬성,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,② 찬성,남,31,3
3,⑤ 매우 반대,② 찬성,④ 반대,⑤ 매우 반대,① 매우 찬성,④ 반대,① 매우 찬성,① 매우 찬성,⑤ 매우 반대,③ 중립,남,51,4
4,③ 중립,③ 중립,③ 중립,③ 중립,② 찬성,③ 중립,③ 중립,③ 중립,③ 중립,③ 중립,여,38,3
5,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,③ 중립,여,47,3
6,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,① 매우 찬성,⑤ 매우 반대,② 찬성,① 매우 찬성,② 찬성,여,21,2
7,② 찬성,① 매우 찬성,② 찬성,② 찬성,④ 반대,③ 중립,③ 중립,② 찬성,③ 중립,① 매우 찬성,남,32,3
8,④ 반대,② 찬성,⑤ 매우 반대,④ 반대,④ 반대,④ 반대,④ 반대,① 매우 찬성,① 매우 찬성,① 매우 찬성,여,52,4
9,⑤ 매우 반대,③ 중립,⑤ 매우 반대,⑤ 매우 반대,① 매우 찬성,② 찬성,② 찬성,② 찬성,④ 반대,① 매우 찬성,남,56,5


비수치화 되어 있는 응답 결과를 숫자 형태의 데이터값으로 변환해 줍니다.

In [3]:
# replace all redundant text to interger values

answerData.replace(["① 매우 찬성","② 찬성", "③ 중립", "④ 반대", "⑤ 매우 반대", "남", "여"], [1, 2, 3, 4, 5, "M", "F"], inplace = True)

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,3,1,2,4,2,1,3,2,3,2,M,40,2
1,1,3,2,2,5,4,2,4,3,1,F,25,2
2,2,5,3,2,1,4,2,3,5,2,M,31,3
3,5,2,4,5,1,4,1,1,5,3,M,51,4
4,3,3,3,3,2,3,3,3,3,3,F,38,3
5,2,4,2,4,2,4,2,4,2,3,F,47,3
6,1,4,2,3,5,1,5,2,1,2,F,21,2
7,2,1,2,2,4,3,3,2,3,1,M,32,3
8,4,2,5,4,4,4,4,1,1,1,F,52,4
9,5,3,5,5,1,2,2,2,4,1,M,56,5


## 응답 유사도 측정

모든 개인과 개인 간의 응답 유사도를 저장할 (데이터 수)^2 사이즈의 행렬인 diffMatrix를 새로 만듭니다.
기본적으로는 NaN (값이 없는 상태)로 초기화합니다.

In [4]:
# create a new dataframe to save values
# this will be very time and cost extensive, might be viable to change this part
# will be durable for several hundred calculations though...?

diffMatrix = pd.DataFrame(np.nan, index=range(dataCount), columns=range(dataCount), dtype='float')

각 개인과 개인의 응답 결과를 비교하여, 각 질문에 대한 총 응답 차이값 = Sum(|개별 응답 차|) 값을 저장합니다.
i번째 응답자와 j번째 응답자의 응답 차이값은 diffMatrix의 i행 j열에 저장됩니다. (j행 i열의 값과 동일합니다.)
자기 자신과의 응답차는 계산하지 않도록 짜 두어, 대각선으로 NaN 값이 나옵니다.

(50인 데이터에서는 수 초 내에 결과가 나오지만, 실제 데이터에서는 응답차 값을 계산하는 것에 상당한 시간이 소요될 것으로 예상됩니다.
 현재 모델에서는 개인과 개인 간의 총 응답차 값을 모두 계산해야 하기 때문에 이 부분이 필수적이라는 점 또한 유의해 주시기 바랍니다.)
 
(09/15 수정: 연령 및 성별 차이를 반영하였습니다. 응답차가 최우선적으로 반영되지만, 성별/연령이 순차적으로 반영될 수 있게 성별은 0.1값, 연령은 0.001값으로 계산합니다.)

In [5]:
for i in range(dataCount):
    for j in range(i, dataCount):
        tempVal=0;
        if (i==j):
            diffMatrix.iloc[i][j] = np.nan
            continue
        for k in range(10):
            # iterates through 10 for the number of questions in the dataframe
            tempVal += answerData.iloc[i][k]-answerData.iloc[j][k]
            
        tempVal = abs(tempVal)
        # assign values for gender/age difference
        if answerData.iloc[i]["gender"] != answerData.iloc[j]["gender"]:
            tempVal += 0.1
            
        tempVal += abs(answerData.iloc[i]["age"]-answerData.iloc[j]["age"])*0.001
        
        # Assign same value to the flipped index to save computation time
        diffMatrix.iloc[i][j] = tempVal
        diffMatrix.iloc[j][i] = tempVal
        
diffMatrix

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,NaN,4.115,6.009,8.011,6.102,6.107,3.119,0.008,7.112,7.016,...,5.005,24.025,19.016,1.117,0.108,4.118,7.020,11.001,7.015,9.005
1,4.115,NaN,2.106,4.126,2.013,2.022,1.004,4.107,3.027,3.131,...,1.120,20.140,15.131,3.032,4.023,0.033,3.105,7.114,11.100,5.120
2,6.009,2.106,NaN,2.020,0.107,0.116,3.110,6.001,1.121,1.025,...,1.014,18.034,13.025,5.126,6.117,2.127,1.011,5.008,13.006,3.014
3,8.011,4.126,2.020,NaN,2.113,2.104,5.130,8.019,1.101,1.005,...,3.006,16.014,11.005,7.106,8.103,4.107,1.031,3.012,15.026,1.006
4,6.102,2.013,0.107,2.113,NaN,0.009,3.017,6.106,1.014,1.118,...,1.107,18.127,13.118,5.019,6.010,2.020,1.118,5.101,13.113,3.107
5,6.107,2.022,0.116,2.104,0.009,NaN,3.026,6.115,1.005,1.109,...,1.102,18.118,13.109,5.010,6.001,2.011,1.127,5.108,13.122,3.102
6,3.119,1.004,3.110,5.130,3.017,3.026,NaN,3.111,4.031,4.135,...,2.124,21.144,16.135,2.036,3.027,1.037,4.101,8.118,10.104,6.124
7,0.008,4.107,6.001,8.019,6.106,6.115,3.111,NaN,7.120,7.024,...,5.013,24.033,19.024,1.125,0.116,4.126,7.012,11.007,7.007,9.013
8,7.112,3.027,1.121,1.101,1.014,1.005,4.031,7.120,NaN,0.104,...,2.107,17.113,12.104,6.005,7.004,3.006,0.132,4.113,14.127,2.107
9,7.016,3.131,1.025,1.005,1.118,1.109,4.135,7.024,0.104,NaN,...,2.011,17.009,12.000,6.101,7.108,3.102,0.036,4.017,14.031,2.011


diffMatrix 내에 저장되어 있는 모든 값들의 중간값을 diffMedian 변수에 저장합니다.
50인 테스트 데이터셋의 경우, 중간값은 5.111에서 형성되었습니다.

In [6]:
# Calculate the median value of all the diff values (currently, it's 5)
diffMedian = diffMatrix.stack().median()

print(diffMedian)

5.111


## 매칭 알고리즘 - 전체 매칭: 2인 기준 (09/15 추가) 

(09/15 추가)

위에 작성한 코드는 기존의 0열(0번 응답자)부터 측정하는 방식으로 진행하였으나, 이 경우에는 참가자 전체에서 가장 유사도/차이도 순서가 높은 순서로 구현합니다. 참가자 전체에 대한 경우의 수를 파악해야 하므로, **시간이 오래 소요될 수 있음**을 유의해 주시기 바랍니다.

전체 모든 가능한 조합 내에서 가장 작은 응답차(성별, 연령 등 포함)를 가진 조합부터 출력합니다.
출력 방식은 ((조합 대상 1, 조합 대상 2), 조합 대상 1의 현재 매칭의 유사도 순위, 조합 대상 2의 현재 매칭의 유사도 순위, 응답차 값) 입니다.

(응답차 값이 긴 소수점으로 나오는 경우는 컴퓨터 연산 상의 소수점 계산 오류로 인한 것이니, 무시하셔도 됩니다.)

조합 대상 a의 현재 매칭의 유사도 순위란, a에게 가능한 모든 조합 중 얼마나 유사한 생각을 가진 사람과 매칭이 되었는가입니다. (전체 풀에서 자신과 더 생각이 유사한 사람이 있지만, 그 사람에게 더 생각이 유사한 사람이 이미 있어서 매칭이 되어 버린 경우입니다.)

(앞선 과정과 유사하거나 같은 기능을 하는 변수들은 _all 을 붙여서 구분합니다)

In [10]:
samePairs_all = []

# Save the number of times each index has been matched
matchSame_all = [0]*dataCount

# Copy the entire diffMatrix values
sameCount_matrix = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    sameCount_matrix[i][i] = 100

while np.min(sameCount_matrix) < diffMedian and np.sum(matchSame_all) < 2*dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmin(sameCount_matrix), (dataCount, dataCount))
    
    if matchSame_all[tempInd[0]] >= 2 or matchSame_all[tempInd[1]] >= 2:
        sameCount_matrix[tempInd[0],tempInd[1]] = 100
        continue
    
    tempRank_0 = (sameCount_matrix[tempInd[0]] == 100).sum() 
    tempRank_1 = (sameCount_matrix[tempInd[1]] == 100).sum() 
    
    samePairs_all.append([tempInd, tempRank_0, tempRank_1, np.min(sameCount_matrix)])
    sameCount_matrix[tempInd[0],tempInd[1]] = 100
    sameCount_matrix[tempInd[1],tempInd[0]] = 100
    
    # increment counts 
    matchSame_all[tempInd[0]] += 1
    matchSame_all[tempInd[1]] += 1
    
            
print(samePairs_all)

[[(1, 23), 1, 1, 0.0], [(27, 44), 1, 1, 0.0], [(25, 48), 1, 1, 0.001], [(10, 27), 1, 2, 0.002], [(10, 44), 2, 2, 0.002], [(17, 21), 1, 1, 0.002], [(24, 40), 1, 1, 0.002], [(8, 12), 1, 1, 0.003], [(13, 16), 1, 1, 0.003], [(15, 32), 1, 1, 0.005], [(18, 33), 1, 1, 0.005], [(26, 47), 1, 1, 0.007], [(0, 7), 1, 1, 0.008], [(4, 5), 1, 1, 0.009000000000000001], [(4, 28), 2, 1, 0.012], [(7, 14), 2, 1, 0.013000000000000001], [(9, 20), 1, 1, 0.013000000000000001], [(1, 36), 2, 1, 0.014], [(23, 36), 2, 2, 0.014], [(2, 31), 1, 1, 0.016], [(31, 32), 2, 2, 0.016], [(37, 39), 1, 1, 0.018000000000000002], [(24, 35), 2, 1, 0.019], [(39, 43), 2, 1, 0.02], [(0, 14), 2, 2, 0.021], [(5, 28), 2, 2, 0.021], [(35, 40), 2, 2, 0.021], [(20, 46), 2, 1, 0.023], [(9, 46), 2, 2, 0.036000000000000004], [(2, 15), 3, 3, 0.037], [(37, 43), 2, 2, 0.038], [(17, 18), 2, 2, 0.1], [(21, 33), 3, 2, 0.10300000000000001], [(3, 49), 2, 1, 1.006], [(22, 48), 1, 2, 1.011], [(22, 25), 2, 2, 1.012], [(8, 29), 7, 8, 1.016], [(12, 29)

다른생각 매칭 또한 같은 원리로

In [11]:
diffPairs_all = []

# Save the number of times each index has been matched
matchDiff_all = [0]*dataCount

# Copy the entire diffMatrix values
diffCount_matrix = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    diffCount_matrix[i][i] = -100

while np.max(diffCount_matrix) > diffMedian and np.sum(matchDiff_all) < 2*dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmax(diffCount_matrix), (dataCount, dataCount))
    
    if matchDiff_all[tempInd[0]] >= 2 or matchDiff_all[tempInd[1]] >= 2:
        diffCount_matrix[tempInd[0],tempInd[1]] = -100
        continue
    
    tempRank_0 = (diffCount_matrix[tempInd[0]] == -100).sum() 
    tempRank_1 = (diffCount_matrix[tempInd[1]] == -100).sum() 
    
    diffPairs_all.append([tempInd, tempRank_0, tempRank_1, np.max(diffCount_matrix)])
    diffCount_matrix[tempInd[0],tempInd[1]] = -100
    diffCount_matrix[tempInd[1],tempInd[0]] = -100
    
    # increment counts 
    matchDiff_all[tempInd[0]] += 1
    matchDiff_all[tempInd[1]] += 1
    
            
print(diffPairs_all)

[[(30, 41), 1, 1, 34.14], [(25, 41), 1, 2, 31.041], [(30, 42), 2, 1, 29.131], [(25, 42), 2, 2, 26.032], [(38, 48), 3, 3, 21.035], [(22, 38), 3, 4, 20.024], [(34, 48), 3, 4, 19.04], [(22, 34), 4, 4, 18.029], [(16, 47), 5, 5, 14.017], [(13, 47), 5, 6, 14.014], [(16, 26), 6, 5, 14.01], [(13, 26), 6, 6, 14.007], [(19, 49), 7, 8, 11.107999999999999], [(3, 19), 9, 8, 10.113999999999999], [(10, 49), 8, 10, 9.105], [(3, 27), 10, 9, 8.103], [(8, 14), 10, 10, 7.133], [(10, 46), 10, 10, 7.13], [(12, 14), 10, 11, 7.13], [(27, 46), 10, 11, 7.127999999999999], [(7, 8), 10, 11, 7.119999999999999], [(7, 12), 11, 11, 7.117], [(9, 44), 11, 13, 7.108], [(20, 44), 12, 14, 7.1049999999999995], [(0, 9), 13, 15, 7.016], [(0, 20), 16, 16, 7.003], [(15, 37), 18, 18, 5.149], [(32, 37), 18, 19, 5.143999999999999], [(11, 28), 18, 18, 5.132], [(15, 39), 19, 18, 5.130999999999999], [(2, 43), 18, 18, 5.1259999999999994], [(32, 39), 19, 19, 5.1259999999999994], [(4, 11), 18, 19, 5.119999999999999]]


매칭 결과를 확인하기 위해 표로 정리했습니다. 50인 데이터를 기준으로, 같은생각/다른생각 모두 매칭되지 않은 경우는 없었습니다.

In [12]:
matchData=pd.DataFrame(
    {'같은생각 매칭 수':matchSame_all, '다른생각 매칭 수':matchDiff_all, '합':np.add(matchSame_all,matchDiff_all)})

matchData

,같은생각 매칭 수,다른생각 매칭 수,합
0,2,2,4
1,2,0,2
2,2,1,3
3,1,2,3
4,2,1,3
5,2,0,2
6,2,0,2
7,2,2,4
8,2,2,4
9,2,2,4


## 매칭 알고리즘 - 전체 매칭: 1인 기준 (09/15 추가) 

위와 동일한 결과를 기준으로, 같은생각/다른생각에 대하여 각 1인씩만 매칭하는 알고리즘으로 변형시켜 보았습니다.

(앞선 과정과 유사하거나 같은 기능을 하는 변수들은 _all_1 을 붙여서 구분합니다)

In [13]:
samePairs_all_1 = []

# Save the number of times each index has been matched
matchSame_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
sameCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    sameCount_matrix_1[i][i] = 100

while np.min(sameCount_matrix_1) < diffMedian and np.sum(matchSame_all_1) < dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmin(sameCount_matrix_1), (dataCount, dataCount))
    
    if matchSame_all_1[tempInd[0]] >= 1 or matchSame_all_1[tempInd[1]] >= 1:
        sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
        continue
    
    tempRank_0 = (sameCount_matrix_1[tempInd[0]] == 100).sum() 
    tempRank_1 = (sameCount_matrix_1[tempInd[1]] == 100).sum() 
    
    samePairs_all_1.append([tempInd, tempRank_0, tempRank_1, np.min(sameCount_matrix_1)])
    sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
    sameCount_matrix_1[tempInd[1],tempInd[0]] = 100
    
    # increment counts 
    matchSame_all_1[tempInd[0]] += 1
    matchSame_all_1[tempInd[1]] += 1
    
            
print(samePairs_all_1)

[[(1, 23), 1, 1, 0.0], [(27, 44), 1, 1, 0.0], [(25, 48), 1, 1, 0.001], [(17, 21), 1, 1, 0.002], [(24, 40), 1, 1, 0.002], [(8, 12), 1, 1, 0.003], [(13, 16), 1, 1, 0.003], [(15, 32), 1, 1, 0.005], [(18, 33), 1, 1, 0.005], [(26, 47), 1, 1, 0.007], [(0, 7), 1, 1, 0.008], [(4, 5), 1, 1, 0.009000000000000001], [(9, 20), 1, 1, 0.013000000000000001], [(2, 31), 1, 1, 0.016], [(37, 39), 1, 1, 0.018000000000000002], [(36, 45), 3, 1, 0.019], [(28, 29), 3, 3, 0.042], [(11, 43), 1, 3, 0.101], [(10, 14), 5, 5, 0.131], [(3, 49), 2, 1, 1.006], [(35, 46), 15, 14, 2.004], [(34, 38), 3, 1, 2.005], [(19, 22), 15, 5, 4.101], [(41, 42), 1, 2, 5.009]]


다른생각 매칭 또한 같은 원리로 진행했습니다.

In [14]:
diffPairs_all_1 = []

# Save the number of times each index has been matched
matchDiff_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
diffCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    diffCount_matrix_1[i][i] = -100

while np.max(diffCount_matrix_1) > diffMedian and np.sum(matchDiff_all_1) < 1*dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmax(diffCount_matrix_1), (dataCount, dataCount))
    
    if matchDiff_all_1[tempInd[0]] >= 1 or matchDiff_all_1[tempInd[1]] >= 1:
        diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
        continue
    
    tempRank_0 = (diffCount_matrix_1[tempInd[0]] == -100).sum() 
    tempRank_1 = (diffCount_matrix_1[tempInd[1]] == -100).sum() 
    
    diffPairs_all_1.append([tempInd, tempRank_0, tempRank_1, np.max(diffCount_matrix_1)])
    diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
    diffCount_matrix_1[tempInd[1],tempInd[0]] = -100
    
    # increment counts 
    matchDiff_all_1[tempInd[0]] += 1
    matchDiff_all_1[tempInd[1]] += 1
    
            
print(diffPairs_all_1)

[[(30, 41), 1, 1, 34.14], [(25, 42), 2, 2, 26.032], [(38, 48), 3, 3, 21.035], [(22, 34), 4, 4, 18.029], [(16, 47), 5, 5, 14.017], [(13, 26), 6, 6, 14.007], [(19, 49), 7, 8, 11.107999999999999], [(3, 27), 10, 9, 8.103], [(8, 14), 10, 10, 7.133], [(10, 46), 10, 10, 7.13], [(7, 12), 11, 11, 7.117], [(9, 44), 11, 13, 7.108], [(0, 20), 16, 16, 7.003], [(15, 37), 18, 18, 5.149], [(11, 28), 18, 18, 5.132], [(2, 43), 18, 18, 5.1259999999999994], [(32, 39), 19, 19, 5.1259999999999994]]


In [ ]:
np.savetxt("samePairs_1.csv", samePairs_all_1, delimiter=",")
np.savetxt("diffPairs_1.csv", diffPairs_all_1, delimiter=",")

매칭 결과를 확인하기 위해 표로 정리했습니다. 50인 데이터를 기준으로, 같은생각/다른생각 모두 매칭되지 않은 경우가 1명 존재합니다.

중간값 등의 도입으로 인해, 같은생각/다른생각 둘 중 한 가지 이상이 매칭되지 않는 경우가 다수 존재합니다.

In [15]:
matchData_1=pd.DataFrame(
    {'같은생각 매칭 수':matchSame_all_1, '다른생각 매칭 수':matchDiff_all_1, '합':np.add(matchSame_all_1,matchDiff_all_1)})

matchData_1

,같은생각 매칭 수,다른생각 매칭 수,합
0,1,1,2
1,1,0,1
2,1,1,2
3,1,1,2
4,1,0,1
5,1,0,1
6,0,0,0
7,1,1,2
8,1,1,2
9,1,1,2


## 매칭 알고리즘 - 행별 계산 (09/15 기준 미사용)

매칭을 통해 만들어진 짝 정보를 저장할 수 있는 빈 array를 만듭니다.
같은생각 매칭은 samePairs에, 다른생각 매칭은 diffPairs에 저장합니다.

In [7]:
# set of arrays to save the created pairs

samePairs = []
diffPairs = []

매칭된 횟수를 별도로 저장할 array를 만들고(초기값은 0입니다), 주어진 규칙에 따라 0열부터 매칭을 진행합니다.
현재 찾은 응답차 최소인 대상이 이미 3번 이상 매치가 되었을 때, 그 값을 무시하고 3회 매칭에 성공하거나,
응답차가 diffMedian 값을 초과할 경우, 이미 모든 응답차 값이 중앙값 이상이기 때문에 매칭을 취소합니다.

(09/09 수정: 0열부터 매칭을 진행, 매칭이 될 경우 지속적으로 매칭을 시도하지 않습니다.
상호 매칭을 허용하고, 매칭의 유사도 순위를 함께 출력합니다. (매칭 결과 상 먼저 나오는 응답자 기준))
(09/15 수정: 기존의 3인 매칭 방식을 다시 가져오고, 대신 2인 매칭으로 수정합니다.)

In [8]:
# create separate array to denote how many times it has been assigned to pairs
matchSame = [0]*dataCount

for i in range(dataCount-1):
    # Shallow copy the current row to mutate
    tempArray = list(diffMatrix[i][i+1:dataCount]) #split array to prevent double matches
    
    j = 0
    while j < 2:
        tempSame = np.argmin(tempArray) # Return mutated index of the current minumum diff individual
        if (tempArray[tempSame] > diffMedian):
            # if the current min value is larger than the median Diff, stop searching  
            break
        elif (matchSame[i] >= 2):
            # if the match count for the current row is already over 3, stop searching
            break
        elif (matchSame[tempSame+i+1] < 2):
            # if the current match count for both items are less than 3, provide match
            samePairs.append([i, tempSame+i+1])
            
            matchSame[i] += 1
            matchSame[tempSame+i+1] +=1
            
            tempArray[tempSame] = 100
            # replace tempArray value to a large number that will not be min-ed
            j += 1 
        else: 
            tempArray[tempSame] = 100
            
print(samePairs)
print(matchSame)

[[0, 7], [0, 14], [1, 23], [1, 36], [2, 31], [2, 32], [3, 9], [3, 49], [4, 5], [4, 28], [5, 28], [6, 33], [6, 23], [7, 14], [8, 12], [8, 9], [10, 27], [10, 44], [11, 43], [11, 39], [12, 20], [13, 16], [13, 19], [15, 32], [15, 31], [16, 19], [17, 21], [17, 18], [18, 33], [20, 46], [21, 37], [22, 48], [22, 25], [24, 40], [24, 35], [25, 48], [26, 47], [26, 34], [27, 44], [29, 40], [29, 35], [34, 47], [36, 45], [37, 39], [38, 42], [38, 49], [41, 42], [43, 45]]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2]


같은 방식으로 다른생각 매칭도 진행하였습니다. 2인 매칭으로 수정된 것을 제외하면 동일합니다.

In [9]:
matchDiff = [0]*dataCount 
   
for i in range(dataCount-1):
    # Shallow copy the current row to mutate
    tempArray = list(diffMatrix[i][i+1:dataCount]) #split array to prevent double matches
    k = 0
    while k < 2:
        tempDiff = np.argmax(tempArray) # Return mutated index of the current maximum diff individual
        if (tempArray[tempDiff] < diffMedian):
            # if the current max value is smaller than the median Diff, stop searching  
            break
        elif (matchDiff[i] >= 2):
            # if the match count for the current row is already over 3, stop searching
            break
        elif (matchDiff[tempDiff+i+1] < 2):
            # if the current match count for both items are less than 3, provide match
            diffPairs.append([i, tempDiff+i+1])
            
            matchDiff[i] += 1
            matchDiff[tempDiff+i+1] +=1
            
            tempArray[tempDiff] = -100
            # replace tempArray value to a small number that will not be max-ed
            k += 1 
        else: 
            tempArray[tempDiff] = -100                 
            
print(diffPairs)
print(matchDiff)

[[0, 41], [0, 42], [1, 41], [1, 42], [2, 30], [2, 25], [3, 30], [3, 25], [4, 48], [4, 22], [5, 48], [5, 22], [6, 38], [6, 34], [7, 38], [7, 34], [8, 16], [8, 13], [9, 16], [9, 13], [10, 26], [10, 47], [11, 26], [11, 47], [12, 19], [12, 14], [14, 49], [15, 19], [15, 27], [17, 49], [18, 20], [20, 27], [28, 44], [29, 44], [31, 37], [32, 37], [32, 39], [39, 46], [43, 46]]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 1, 2, 1, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 2, 2, 2]
